<a href="https://colab.research.google.com/github/Aloxxa/r_public/blob/main/%22%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_3%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 задание

Написать на python генератор первых n чисел фибоначи.

In [ ]:
def fibonacci_generator(n):
    a, b = 1, 1
    for i in range(n):
        yield a
        a, b = b, a + b

In [ ]:
n = 10
for num in fibonacci_generator(n):
    print(num)

1
1
2
3
5
8
13
21
34
55


# 2 задание

In [ ]:
!pip install Faker

  Using cached Faker-24.4.0-py3-none-any.whl (1.8 MB)


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from faker import Faker
fake = Faker()

In [ ]:
Department_df = pd.DataFrame({'id': range(1000)})

In [ ]:
Department_df['name'] = list([fake.job() for i in range(1000)])
Department_df['name'].nunique()

518

In [ ]:
Department_df

,id,name
0,0,Community development worker
1,1,Theatre director
2,2,Barrister's clerk
3,3,Politician's assistant
4,4,"Development worker, international aid"
...,...,...
995,995,Copy
996,996,"Designer, interior/spatial"
997,997,Broadcast journalist
998,998,"Doctor, hospital"


In [ ]:
Personal_df = pd.DataFrame({'id': range(1000)})

In [ ]:
Personal_df['Id_head'] = pd.Series(['1', '2', '3', '4', '5', '7', '9', '10', '23', '26', '30', '42', '45', '80']).sample(1000, replace=True, random_state=42).values

In [ ]:
Personal_df['Id_dep'] = pd.Series(['1', '2', '3', '4', '5', '777', '42', '2015', '2077', '2033', '2042']).sample(1000, replace=True, random_state=42).values

In [ ]:
np.random.seed(42)
names = [fake.name() for i in range(1000)]
1000 - len(set(names))
#нужно вручную дописывать имена, если нужно строго 1000 уникальных име

8

In [ ]:
Personal_df['name'] = names

In [ ]:
Personal_df['sal'] = (np.round(np.random.lognormal(mean=10, sigma=0.5, size=1000) / 1000) * 1000).astype(int)
#данное распределние подходит, так как есть хвосты

In [ ]:
Personal_df

,id,Id_head,Id_dep,name,sal
0,0,9,42,Malik Evans,28000
1,1,4,4,Daniel Allen,21000
2,2,45,2042,Robert Black,30000
3,3,30,2015,Shannon Franklin,47000
4,4,10,5,Gary Hall,20000
...,...,...,...,...,...
995,995,10,2033,Thomas Ortiz,19000
996,996,42,2077,Elizabeth Murphy,54000
997,997,80,2,Kyle Kelly,30000
998,998,9,2015,John Schneider,17000


In [ ]:
con = sqlite3.connect('db')

In [ ]:
Personal_df.to_sql('Personal', con, if_exists='replace', index=False, method='multi')

1000

In [ ]:
Department_df.to_sql('Department', con, if_exists='replace', index=False, method='multi')

1000

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
Department_df.columns

Index(['id', 'name'], dtype='object')

In [ ]:
Personal_df.columns

Index(['id', 'Id_head', 'Id_dep', 'name', 'sal'], dtype='object')

1.	Вывести отдел с наибольшим числом сотрудников;

In [ ]:
sql_2_1 = '''

with max_empl as (
  select d.name, count(p.id) as count_empl
  from Department d
  left join Personal p on d.id = p.Id_dep
  group by d.name
)
select name as отдел_с_наибольшим_числом_сотрудников
from max_empl
where count_empl = (select max(count_empl) from max_empl)
'''

In [ ]:
select(sql_2_1)

,отдел_с_наибольшим_числом_сотрудников
0,Theatre director


In [ ]:
sql_2_1_check = '''

with max_empl as (
  select d.name, count(p.id) as count_empl
  from Department d
  left join Personal p on d.id = p.Id_dep
  group by d.name
  order by count(*) desc
  limit 10
)
select *
from max_empl

'''

In [ ]:
select(sql_2_1_check)

,name,count_empl
0,Theatre director,110
1,Politician's assistant,103
2,Stage manager,102
3,Comptroller,86
4,Learning disability nurse,87
5,"Development worker, international aid",84
6,Barrister's clerk,79
7,Visual merchandiser,0
8,Tourism officer,0
9,Tax inspector,0


2.	Вывести список сотрудников, получающих заработную плату выше, чем у руководителя;

In [ ]:
sql_2_2 = '''

select p.name as employee_name
from Personal p
join Personal h on p.Id_head = h.id
where p.sal > h.sal;
'''

In [ ]:
select(sql_2_2)

,employee_name
0,Daniel Allen
1,Robert Black
2,Shannon Franklin
3,Gary Hall
4,Nancy Thompson
...,...
527,Steven Phillips
528,Thomas Ortiz
529,Elizabeth Murphy
530,Kyle Kelly


In [ ]:
sql_2_2_check = '''

select p.name as employee_name, p.sal AS employee_salary, h.name AS head_name, h.sal AS head_salary
from Personal p
join Personal h on p.Id_head = h.id
where p.sal > h.sal;
'''

In [ ]:
select(sql_2_2_check)

,employee_name,employee_salary,head_name,head_salary
0,Daniel Allen,21000,Gary Hall,20000
1,Robert Black,30000,Tony Mckenzie,15000
2,Shannon Franklin,47000,Nathaniel Griffin,16000
3,Gary Hall,20000,Kirk Stanley,17000
4,Nancy Thompson,20000,Tony Mckenzie,15000
...,...,...,...,...
527,Steven Phillips,19000,Gabriel Martinez,11000
528,Thomas Ortiz,19000,Kirk Stanley,17000
529,Elizabeth Murphy,54000,Michael Stephenson,21000
530,Kyle Kelly,30000,Joshua Vincent,20000


#Задание 3

In [1]:
# Base
# -----------------------------------
import random
import numpy as np
import datetime
import pandas as pd
from datetime import datetime, timedelta
# Visualization
# -----------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from wordcloud import WordCloud
import plotly.io as pio
# Configuration
# -----------------------------------
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.style.use('dark_background')

In [2]:
!gdown --id 1EvqGPgMTTTV0x18hRa6yXS-VZGvNyh7I

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1EvqGPgMTTTV0x18hRa6yXS-VZGvNyh7I
To: /content/inn_info_public.csv
100% 4.51M/4.51M [00:00<00:00, 42.1MB/s]


In [3]:
INN_data = pd.read_csv('/content/inn_info_public.csv')

In [4]:
!gdown --id 1oes1YUk4j0LgJiUFlSkP2iNIrBdI6Azd

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1oes1YUk4j0LgJiUFlSkP2iNIrBdI6Azd
From (redirected): https://drive.google.com/uc?id=1oes1YUk4j0LgJiUFlSkP2iNIrBdI6Azd&confirm=t&uuid=be8e4cd3-814e-410f-b297-aaf108e03a1f
To: /content/pays.csv
100% 135M/135M [00:02<00:00, 57.1MB/s]


In [5]:
PAYS_data = pd.read_csv('/content/pays.csv')

In [6]:
df_inn = INN_data.copy()
df_pays = PAYS_data.copy()

**EDA df_inn**

In [9]:
path = '/content/inn_info_public.csv'

def load(path, info = True):

    import io

    if len(path.split(".csv")) > 1:
        read = pd.read_csv(path)
    elif len(path.split(".xlsx")) > 1:
        read = pd.read_excel(path)

    if info:
        if len(read) > 0:
            print("# Data imported!")
            print("# ------------------------------------", "\n")

            print("# DIMENSIONS -------------------------")
            print("Observation:", read.shape[0], "Column:", read.shape[1], "\n")

            print("# DTYPES -----------------------------")
            if len(read.select_dtypes("object").columns) > 0:
                print("Object Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("object").columns), "\n",
                      read.select_dtypes("object").columns.tolist(), "\n")

            if len(read.select_dtypes("integer").columns) > 0:
                print("Integer Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("integer").columns), "\n",
                      read.select_dtypes("integer").columns.tolist(), "\n")

            if len(read.select_dtypes("float").columns) > 0:
                print("Float Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("float").columns), "\n",
                      read.select_dtypes("float").columns.tolist(), "\n")

            if len(read.select_dtypes("bool").columns) > 0:
                print("Bool Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("bool").columns), "\n",
                      read.select_dtypes("bool").columns.tolist(), "\n")

            print("# MISSING VALUE ---------------------")
            print("Are there any missing values? \n ", np.where(read.isnull().values.any() == False,
                                                            "No missing value!", "Data includes missing value!"), "\n")

            buf = io.StringIO()
            read.info(buf=buf)
            info = buf.getvalue().split('\n')[-2].split(":")[1].strip()
            print("# MEMORY USAGE ---------------------- \n", info)

        else:
            print("# Data did not import!")

    return read

t = load(path, info = True)
t.head()

# Data imported!
# ------------------------------------ 

# DIMENSIONS -------------------------
Observation: 240069 Column: 4 

# DTYPES -----------------------------
Integer Variables: 
 # of Variables: 3 
 ['hash_inn', 'okved2', 'region'] 

Bool Variables: 
 # of Variables: 1 
 ['is_public'] 

# MISSING VALUE ---------------------
Are there any missing values? 
  No missing value! 

# MEMORY USAGE ---------------------- 
 5.7 MB


,hash_inn,okved2,region,is_public
0,61058,34,86,True
1,8311,18,86,True
2,130273,-1,86,False
3,64081,43,86,True
4,218005,12,86,True


In [17]:
data_duplicates_rows = df_inn[df_inn.duplicated()]
data_duplicates_rows

,hash_inn,okved2,region,is_public


**EDA df_pays**

In [10]:
path = '/content/pays.csv'

def load(path, info = True):

    import io

    if len(path.split(".csv")) > 1:
        read = pd.read_csv(path)
    elif len(path.split(".xlsx")) > 1:
        read = pd.read_excel(path)

    if info:
        if len(read) > 0:
            print("# Data imported!")
            print("# ------------------------------------", "\n")

            print("# DIMENSIONS -------------------------")
            print("Observation:", read.shape[0], "Column:", read.shape[1], "\n")

            print("# DTYPES -----------------------------")
            if len(read.select_dtypes("object").columns) > 0:
                print("Object Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("object").columns), "\n",
                      read.select_dtypes("object").columns.tolist(), "\n")

            if len(read.select_dtypes("integer").columns) > 0:
                print("Integer Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("integer").columns), "\n",
                      read.select_dtypes("integer").columns.tolist(), "\n")

            if len(read.select_dtypes("float").columns) > 0:
                print("Float Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("float").columns), "\n",
                      read.select_dtypes("float").columns.tolist(), "\n")

            if len(read.select_dtypes("bool").columns) > 0:
                print("Bool Variables:", "\n", "# of Variables:",
                      len(read.select_dtypes("bool").columns), "\n",
                      read.select_dtypes("bool").columns.tolist(), "\n")

            print("# MISSING VALUE ---------------------")
            print("Are there any missing values? \n ", np.where(read.isnull().values.any() == False,
                                                            "No missing value!", "Data includes missing value!"), "\n")

            buf = io.StringIO()
            read.info(buf=buf)
            info = buf.getvalue().split('\n')[-2].split(":")[1].strip()
            print("# MEMORY USAGE ---------------------- \n", info)

        else:
            print("# Data did not import!")

    return read

t = load(path, info = True)
t.head()

# Data imported!
# ------------------------------------ 

# DIMENSIONS -------------------------
Observation: 5430150 Column: 5 

# DTYPES -----------------------------
Integer Variables: 
 # of Variables: 4 
 ['hash_inn_kt', 'hash_inn_dt', 'week', 'count'] 

Float Variables: 
 # of Variables: 1 
 ['sum'] 

# MISSING VALUE ---------------------
Are there any missing values? 
  Data includes missing value! 

# MEMORY USAGE ---------------------- 
 207.1 MB


,hash_inn_kt,hash_inn_dt,week,count,sum
0,0,0,1,4,38399.6
1,0,0,5,2,399.6
2,0,0,7,2,79.6
3,0,0,9,2,239.6
4,0,0,12,2,79.6


In [13]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)

        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

        # Return the dataframe with missing information
        return mis_val_table_ren_columns


missing_values_table(df_pays)

Your selected dataframe has 5 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
sum,1125,0.0


In [14]:
df_pays = df_pays.dropna()

In [16]:
data_duplicates_rows = df_pays[df_pays.duplicated()]
data_duplicates_rows

,hash_inn_kt,hash_inn_dt,week,count,sum


**Работа с датафреймами**

In [18]:
data = pd.merge(df_pays, df_inn, left_on='hash_inn_kt', right_on='hash_inn', how='inner')

In [19]:
df = data.copy()

In [20]:
df

,hash_inn_kt,hash_inn_dt,week,count,sum,hash_inn,okved2,region,is_public
0,0,0,1,4,38399.6,0,-1,84,False
1,0,0,5,2,399.6,0,-1,84,False
2,0,0,7,2,79.6,0,-1,84,False
3,0,0,9,2,239.6,0,-1,84,False
4,0,0,12,2,79.6,0,-1,84,False
...,...,...,...,...,...,...,...,...,...
5429020,260511,77935,5,2,399.6,260511,12,84,True
5429021,260511,77935,12,2,439.6,260511,12,84,True
5429022,260511,77935,14,2,7.6,260511,12,84,True
5429023,260511,77935,15,2,79.6,260511,12,84,True


In [22]:
df_train = df[df['is_public'] == 0]

In [23]:
df_test = df[df['is_public'] == 1]

In [ ]:
X_train = df_train[['count', 'sum', 'region',]]
y_train = df_train['okved2']
X_test = df_test[['count', 'sum', 'region',]]
y_test = df_test['okved2']

In [24]:
from sklearn.ensemble import RandomForestClassifier
# https://colab.research.google.com/drive/16Ng4qvaDml21rUbLKL67sEHSnL2cU4Qw?usp=sharing

In [ ]:
model = RandomForestClassifier(random_state=42)

In [ ]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
pred_okved2 = model.predict(X_test)

In [ ]:
pred_okved2

array([52, 12, 12, ..., 12, 12, 14])

In [ ]:
df_test['okved2'] = pred_okved2

<ipython-input-28-16e4624ea753>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['okved2'] = pred_okved2


In [ ]:
df_test

,hash_inn_kt,hash_inn_dt,week,count,sum,hash_inn,okved2,region,is_public
0,0,0.0,1.0,4.0,38399.600,0,52,84,False
1,0,0.0,5.0,2.0,399.600,0,12,84,False
2,0,0.0,7.0,2.0,79.600,0,12,84,False
3,0,0.0,9.0,2.0,239.600,0,12,84,False
4,0,0.0,12.0,2.0,79.600,0,12,84,False
...,...,...,...,...,...,...,...,...,...
135748,6947,91985.0,13.0,2.0,0.896,6947,41,84,False
135749,6947,200061.0,5.0,2.0,50.480,6947,14,84,False
135766,6951,13043.0,10.0,1.0,2.800,6951,12,85,False
135767,6951,251325.0,6.0,1.0,1.184,6951,12,85,False


In [ ]:
final_df = pd.concat([df_train, df_test])

In [ ]:
final_df_ds_sber = pd.DataFrame({'hash_inn': df_test['hash_inn_kt'], 'Industry': pred_okved2})
#неверно (hash_inn без конкретной Industry)

In [ ]:
final_df_ds_sber

,hash_inn,Industry
0,0,52
1,0,12
2,0,12
3,0,12
4,0,12
...,...,...
135748,6947,41
135749,6947,14
135766,6951,12
135767,6951,12


In [ ]:
ind_counts = final_df_ds_sber.groupby('hash_inn')['Industry'].value_counts()
#нужно найти Industry, которая чаще всего встречалась (для каждого hash_inn)

In [ ]:
max_ind = ind_counts.groupby(level=0).idxmax()

In [ ]:
final_csv = pd.DataFrame(max_ind.tolist(), columns=['hash_inn', 'Industry'])

In [ ]:
mask = df['okved2'] == -1
df.loc[mask, 'okved2'] = df[mask]['hash_inn'].map(final_csv.set_index('hash_inn')['Industry'])
#подставим в исходный df полученные okved2

In [ ]:
df['okved2'].value_counts(normalize=True)

12    0.482437
34    0.119718
39    0.064551
4     0.058511
14    0.032820
        ...   
31    0.000052
51    0.000022
35    0.000015
2     0.000007
63    0.000007
Name: okved2, Length: 77, dtype: float64

In [ ]:
hash_inn_unknown = df[df['okved2'] == -1]['hash_inn'].nunique()
hash_inn_unknown
#проверим, все ли hash_inn_unknown получили okved2

0

In [ ]:
final_csv.to_csv('AlexChukavin-24032024-Sberbank-Industry.csv')

In [ ]:
from google.colab import files
files.download("AlexChukavin-24032024-Sberbank-Industry.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>